![](images/conversation_tree.png)

::: {.callout-tip collapse="true"}
## Setting up

For this tutorial, you will need to install a few libraries and an llm. The llm will be the one that organises the chat and the one you talk to. If you use a locally hosted model (you can!) you can skip the setting up of the api key [Click here to get a key visit](https://console.groq.com/keys).

For this tutorial, I have choosen Kimi-K2 hosted by Groq. This is pretty cheap, very fast, and pretty smart! 

::: {.callout-note icon=false appearance="simple" collapse="true"}
## python library requirements
I like to use uv to install my libraries.


In [33]:
#| output: false
#| code-fold: false
#| code-summary: ""
!uv pip install dspy networkx pyvis anytree

Using Python 3.12.10 environment at: /home/maxime/Projects/maximerivest-blog/.venv
Resolved 88 packages in 293ms                                        
Prepared 1 package in 24ms                                               
Installed 1 package in 3ms                                  
 + anytree==2.13.0


:::

::: {.callout-note icon=false appearance="simple" collapse="true"}
## api key setup
I generally setup my key permanently but you can also do this to set it up just for here and now.

::: {.callout-note icon=false appearance="simple" collapse="true"}
## Make GROQ_API_KEY permanent

###### Linux / macOS
Append to your shell start-up file (pick the one you actually use):

```bash
echo "export GROQ_API_KEY='gsk_[REDACTED]'" >> ~/.bashrc
# or ~/.zshrc, ~/.profile, etc.
source ~/.bashrc   # reload once
```

###### Windows – CMD
```cmd
setx GROQ_API_KEY "gsk_[REDACTED]"
```
Close and reopen the terminal.

###### Windows – PowerShell
```powershell
[Environment]::SetEnvironmentVariable("GROQ_API_KEY", "gsk_[REDACTED]", "User")
```
Refresh with `refreshenv` or open a new window.
:::

In [ ]:
#| output: false
#| eval: false
# import os
# os.environ["GROQ_API_KEY"] = "gsk_[REDACTED]"

:::

:::

To build an automaticaly branching chat we will need many pieces.

1. We need a data structure that will old the chat-tree.
2. We need a conversation router that will decide where the users prompt get connected in the tree
3. We need an interface for the user to chat
4. We need a way to build back the relevant conversation trace into an llm ready conversation


::: {.callout-note appearance="simple"}

In this, tutorial we focus on the AI programming part. As such, our implementation will be far from optimal computationally.

:::

Let's first think of the data structure. For this tutorial, we will keep it simple. It will
be a list of 'turns', each turns will point to their parent turn.

In [46]:
import pydantic
from typing import List, Optional, Dict

class Turn(pydantic.BaseModel):
    turn_id: int
    parent_turn_id: Optional[int]
    user: str
    assistant: str
    children_ids: List[int] = pydantic.Field(default_factory=list)

turn_i = Turn(
    turn_id = 0, 
    parent_turn_id = None, 
    user = "Help me understand gravity.",
    assistant = "Gravity is the force that pulls any two pieces of matter toward each other. On Earth, it gives objects weight and keeps us on the ground. In space, it keeps the Moon orbiting Earth and the planets orbiting the Sun. According to Einstein, massive objects actually bend the fabric of space-time, and what we feel as gravity is simply objects following the curved paths created by that bending."
)


In [47]:
print(turn_i.model_dump_json(indent=2))

{
  "turn_id": 0,
  "parent_turn_id": null,
  "user": "Help me understand gravity.",
  "assistant": "Gravity is the force that pulls any two pieces of matter toward each other. On Earth, it gives objects weight and keeps us on the ground. In space, it keeps the Moon orbiting Earth and the planets orbiting the Sun. According to Einstein, massive objects actually bend the fabric of space-time, and what we feel as gravity is simply objects following the curved paths created by that bending.",
  "children_ids": []
}


We will be using the tree in the image above to develop our programs.

Here we are creating a conversation tree object to help us find tips, roots, and collect turns from a tip until a certain depth. If you follow along, you will need to copy paste and run them but you do not need to understand them to understand the tutorial.

In [55]:
#| code-fold: true
#| code-summary: 'defining the ConversationTree object'
import pydantic
from typing import List, Optional, Dict

class Turn(pydantic.BaseModel):
    turn_id: int
    parent_turn_id: Optional[int]
    user: str
    assistant: str
    children_ids: List[int] = pydantic.Field(default_factory=list)

class ConversationTree:
    def __init__(self):
        self.turns: Dict[int, Turn] = {}

    def add_turn(self, turn: Turn):
        self.turns[turn.turn_id] = turn
        if turn.parent_turn_id is not None:
            parent_turn = self.turns[turn.parent_turn_id]
            parent_turn.children_ids.append(turn.turn_id)

    def get_turn(self, turn_id: int) -> Turn:
        return self.turns[turn_id]

    def get_root_turns(self) -> List[Turn]:
        return [turn for turn in self.turns.values() if turn.parent_turn_id is None]

    def get_leaf_turns(self) -> List[Turn]:
        return [turn for turn in self.turns.values() if len(turn.children_ids) == 0]

    def trace_upward(self, turn_id: int, depth: int = 4) -> List[Turn]:
        trace = []
        current = self.get_turn(turn_id)
        while current and len(trace) < depth:
            trace.append(current)
            if current.parent_turn_id is not None:
                current = self.get_turn(current.parent_turn_id)
            else:
                break
        return trace[::-1]  # reverse to get root to leaf order

    def trace_downward(self, turn_id: int, depth: int = 4) -> List[List[Turn]]:
        traces = []

        def dfs(current_id, current_trace):
            if len(current_trace) == depth:
                traces.append(current_trace[:])
                return
            current_turn = self.get_turn(current_id)
            if not current_turn.children_ids:
                traces.append(current_trace[:])
                return
            for child_id in current_turn.children_ids:
                dfs(child_id, current_trace + [self.get_turn(child_id)])

        dfs(turn_id, [self.get_turn(turn_id)])
        return traces


In [57]:
#| code-fold: true
#| code-summary: 'fulling up the ConversationTree object'

conversation_tree = ConversationTree()

conversations = [
    Turn(turn_id=0, parent_turn_id=None, user="Help me understand gravity.", assistant="Gravity is the force..."),
    Turn(turn_id=1, parent_turn_id=0, user="What's the difference between Newton's and Einstein's theories of gravity?", assistant="Newton pictured gravity..."),
    Turn(turn_id=2, parent_turn_id=1, user="Is gravity a force or something else?", assistant="It depends on the theory..."),
    Turn(turn_id=3, parent_turn_id=0, user="you said Gravity is the force that pulls any two pieces of matter, can you show me the formula", assistant="Newton’s universal law..."),
    Turn(turn_id=4, parent_turn_id=None, user="Give me a good recipe for a vegan pasta sauce.", assistant="Creamy Tomato-Basil Vegan Pasta Sauce..."),
    Turn(turn_id=5, parent_turn_id=4, user="For the recipe, I don't like onion can you improve", assistant="Creamy Tomato-Basil Vegan Pasta Sauce (No-Onion Version)..."),
    Turn(turn_id=6, parent_turn_id=None, user="Who coined the word gravity?", assistant="Isaac Newton first used..."),
    Turn(turn_id=7, parent_turn_id=6, user="How old was he?", assistant="Isaac Newton was 44–45 years old..."),
    Turn(turn_id=8, parent_turn_id=7, user="Where did he live?", assistant="He lived in England..."),
]

for conv in conversations:
    conversation_tree.add_turn(conv)

Now that we have a data structure (the turns and tree) we can focus on the interesting part, the conversation router!

The goal of this part is to find the best place in the tree to attach the current user prompt.

We will do that by doing a sort of tournament for each roots (10 at a time) and also the the tips.
The reason for the tournament is that we could have 100s of roots and even more tips and an llm has a context window limit. 

We are doing 10 at a time because llms tend to be better an ranking then scoring according to a rubric [RULER is inspiring me](https://art.openpipe.ai/fundamentals/ruler).

We will do it as follow:

1. Check me relevance of root using first 4 turns.
2. For the 10 most relevant root, check for relevance of tips (roots may have several tips)
3. For the 10 most relevant tips, collect all the conversation traces.
4. Check for the most relevant conversation turn to attach too out of all turns in the short listed conversation traces.

let's collect all roots and get 4 turns

In [102]:
roots = conversation_tree.get_root_turns()
traces = []
for i_root in roots:
    traces.extend(conversation_tree.trace_downward(turn_id=i_root.turn_id, depth=4))

Let's see how many traces we collected

In [109]:
len(traces)

4

In [110]:
traces[0]

[Turn(turn_id=0, parent_turn_id=None, user='Help me understand gravity.', assistant='Gravity is the force...', children_ids=[1, 3]),
 Turn(turn_id=1, parent_turn_id=0, user="What's the difference between Newton's and Einstein's theories of gravity?", assistant='Newton pictured gravity...', children_ids=[2]),
 Turn(turn_id=2, parent_turn_id=1, user='Is gravity a force or something else?', assistant='It depends on the theory...', children_ids=[])]

We could probably show these to the llm but I think we can render that into something a little more readable

In [111]:
def format_trace(trace: List[Turn]) -> str:
    trace_string = ""
    for turn in trace:
        trace_string += "\n\n## User: \n\t" + turn.user + \
                        "\n\n## Assistant: \n\t" + turn.assistant
    return trace_string

In [112]:
print(format_trace(traces[2]))



## User: 
	Give me a good recipe for a vegan pasta sauce.

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce...

## User: 
	For the recipe, I don't like onion can you improve

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce (No-Onion Version)...


Now that we have all conversation segments from root to 4 we are ready to rank them by relevance.

So for this, we need to provide a prompt and the candidate segment and we would like to receive a ranking where one is the best, and a relevance score from 0 to 1, and maybe some sort of temporary trace id so that we know what relevance goes for which trace. let's make a list:

Inputs:

* current user prompt (string)
* first 4 turns of many root chats  (string with chat trace temporary id)

Outputs:

* a sorted list of evaluations which if contains: a rank (int), a trace id (int), a relevance score (float from 0 to 1)

let's turn that into a dspy program.

The first step for our dspy program is to define a class that will ensure we get from the llm exactly what we want

In [120]:
class SegmentEvaluation(pydantic.BaseModel):
    trace_id: int
    relevance: float
    segment_rank: int

Now we will write our program's instructions, inputs and outputs as a DSPy signature

In [121]:
class EvaluateRootSegments(dspy.Signature):
    """Evaluate root conversation segments for relevance to a new prompt.

    For each segment, identify if has topical connection to the user prompt. Consider if the prompt is:
    - A direct follow-up question.
    - A request for clarification.
    - An exploration of a related sub-topic.
    - A completely different subject.
    
    Assign a relevance score from 0.0 (completely irrelevant) to 1.0 (a direct continuation of the topic).
    You will also rank the segments where 1 is the most relevant of the group
    """
    #Inputs
    user_prompt: str = dspy.InputField(desc="The new user prompt to be integrated.")
    segments_to_evaluate: str = dspy.InputField(desc="A stringified list of conversation segments, each with its trace_id and content.")
    
    #Outputs
    evaluations: List[SegmentEvaluation] = dspy.OutputField(desc="A list of evaluations, one for each segment, including detailed reasoning.")

Now to make that signature callable we have to make it into a module. The simplest one is `dspy.Predict`, let's use that.

In [122]:
root_relevance_evaluator = dspy.Predict(EvaluateRootSegments)

We are almost ready to call an AI and get those traces ranked, but we first need to format our traces all in one nice piece of context for the llm.

In [126]:
def format_traces_with_id(traces):
    count = 0
    all_traces_string = ""
    for trace in traces:
        count += 1
        all_traces_string += f"<trace_id = {count}>\n" + \
                                    format_trace(trace)+ \
                             f"\n</trace_id = {count}>\n"
    return all_traces_string 
print(format_traces_with_id(traces))

<trace_id = 1>


## User: 
	Help me understand gravity.

## Assistant: 
	Gravity is the force...

## User: 
	What's the difference between Newton's and Einstein's theories of gravity?

## Assistant: 
	Newton pictured gravity...

## User: 
	Is gravity a force or something else?

## Assistant: 
	It depends on the theory...
</trace_id = 1>
<trace_id = 2>


## User: 
	Help me understand gravity.

## Assistant: 
	Gravity is the force...

## User: 
	you said Gravity is the force that pulls any two pieces of matter, can you show me the formula

## Assistant: 
	Newton’s universal law...
</trace_id = 2>
<trace_id = 3>


## User: 
	Give me a good recipe for a vegan pasta sauce.

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce...

## User: 
	For the recipe, I don't like onion can you improve

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce (No-Onion Version)...
</trace_id = 3>
<trace_id = 4>


## User: 
	Who coined the word gravity?

## Assistant: 
	Isaac Newton first used...

## User

We will be calling an AI now to execute `root_relevance_evaluator`. So we set is up like that:

In [141]:
lm = dspy.LM("groq/moonshotai/kimi-k2-instruct")
dspy.configure(lm = lm)

Connecting to different models and providers in DSPy is very easy. You just have to change `groq/moonshotai/kimi-k2-instruct` for the path to the provider and model you want. Behind the scene, dspy uses litellm so this path is one that would work with litellm^[for instance you could do `gpt-4.1`, or `ollama/<ollama_model>`]

In [134]:
evaluation = root_relevance_evaluator(
    user_prompt = "should I add cake for desert?",
    segments_to_evaluate = format_traces_with_id(traces)
)
root_evals = evaluation.evaluations
root_evals

[SegmentEvaluation(trace_id=1, relevance=0.0, segment_rank=4),
 SegmentEvaluation(trace_id=2, relevance=0.0, segment_rank=3),
 SegmentEvaluation(trace_id=3, relevance=0.15, segment_rank=1),
 SegmentEvaluation(trace_id=4, relevance=0.0, segment_rank=2)]

let's do the same for the tips

In [135]:
tips = conversation_tree.get_leaf_turns()
traces = []
for i_tip in tips:
    traces.append(conversation_tree.trace_upward(turn_id=i_tip.turn_id, depth=4))

print(format_traces_with_id(traces))

<trace_id = 1>


## User: 
	Help me understand gravity.

## Assistant: 
	Gravity is the force...

## User: 
	What's the difference between Newton's and Einstein's theories of gravity?

## Assistant: 
	Newton pictured gravity...

## User: 
	Is gravity a force or something else?

## Assistant: 
	It depends on the theory...
</trace_id = 1>
<trace_id = 2>


## User: 
	Help me understand gravity.

## Assistant: 
	Gravity is the force...

## User: 
	you said Gravity is the force that pulls any two pieces of matter, can you show me the formula

## Assistant: 
	Newton’s universal law...
</trace_id = 2>
<trace_id = 3>


## User: 
	Give me a good recipe for a vegan pasta sauce.

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce...

## User: 
	For the recipe, I don't like onion can you improve

## Assistant: 
	Creamy Tomato-Basil Vegan Pasta Sauce (No-Onion Version)...
</trace_id = 3>
<trace_id = 4>


## User: 
	Who coined the word gravity?

## Assistant: 
	Isaac Newton first used...

## User

In [137]:
evaluation = root_relevance_evaluator(
    user_prompt = "should I add cake for desert?",
    segments_to_evaluate = format_traces_with_id(traces)
)
tip_evals = evaluation.evaluations
tip_evals

[SegmentEvaluation(trace_id=1, relevance=0.0, segment_rank=4),
 SegmentEvaluation(trace_id=2, relevance=0.0, segment_rank=3),
 SegmentEvaluation(trace_id=3, relevance=0.15, segment_rank=1),
 SegmentEvaluation(trace_id=4, relevance=0.0, segment_rank=2)]

In [140]:
best_root_eval = max(root_evals, key=lambda x: x.relevance)
best_tip_eval = max(tip_evals, key=lambda x: x.relevance)

print(best_root_eval)
print(best_tip_eval)

trace_id=3 relevance=0.15 segment_rank=1
trace_id=3 relevance=0.15 segment_rank=1


In [4]:
class ConnectionDecision(pydantic.BaseModel):
    turn_id: int
    attach_at_turn_index: int
    reasoning: str


class ResolveConflict(dspy.Signature):
    user_prompt: str = dspy.InputField()
    candidate_A_info: str = dspy.InputField()
    candidate_B_info: str = dspy.InputField()
    decision: ConnectionDecision = dspy.OutputField()


class FollowUpDecision(pydantic.BaseModel):
    is_follow_up: bool = pydantic.Field(..., description="True if the new prompt is a direct contextual follow-up, otherwise False.")
    reasoning: str = pydantic.Field(..., description="A brief explanation for the decision.")

class CheckForFollowUp(dspy.Signature):
    """Determine if a new prompt is a direct follow-up to the immediate previous conversation turn.

    Focus on pronouns (he, she, it, they), demonstratives (that, this), or questions that only make sense with the preceding context.
    """
    last_turn: str = dspy.InputField(desc="The most recent user-assistant exchange.")
    new_prompt: str = dspy.InputField(desc="The new user prompt.")
    decision: FollowUpDecision = dspy.OutputField(desc="A boolean decision and reasoning.")

In [5]:
class ConversationRouter(dspy.Module):
    def __init__(self):
        super().__init__()
        self.evaluate_segments = dspy.ChainOfThought(EvaluateSegments)
        self.resolve_conflict = dspy.Predict(ResolveConflict)
        # NEW: Add the follow-up checker
        self.check_follow_up = dspy.Predict(CheckForFollowUp)

    def _format_turns(self, turns: List[dict]) -> str:
        return "\n".join([f"{turn['role'].capitalize()}: {turn['content']}" for turn in turns])

    def forward(self, user_prompt: str, conversations: List[dict], last_active_turn: dict = None):
        if not conversations:
            decision = ConnectionDecision(turn_id=0, attach_at_turn_index=-1, reasoning="No existing conversations.")
            return {'decision': decision, 'score': 1.0}

        # --- NEW: Recency / Follow-up Check ---
        if last_active_turn:
            # Context is the last exchange in the most recent turn
            # We take the full message list of the last active turn segment
            last_turn_context = self._format_turns(last_active_turn['messages'])
            follow_up_result = self.check_follow_up(last_turn=last_turn_context, new_prompt=user_prompt)

            if follow_up_result.decision.is_follow_up:
                print(f"🧠 Follow-up Detected: {follow_up_result.decision.reasoning}")
                
                # The parent is the last turn itself, and we attach at its very last message
                attach_index = len(last_active_turn['messages']) - 1
                
                decision = ConnectionDecision(
                    turn_id=last_active_turn['turnID'],
                    attach_at_turn_index=attach_index,
                    reasoning=f"Direct follow-up to the most recent turn. {follow_up_result.decision.reasoning}"
                )
                # Return with a perfect score to ensure it's selected
                return {'decision': decision, 'score': 1.0}
        
        # --- ORIGINAL LOGIC (if no follow-up is detected) ---
        print("🤔 No direct follow-up. Evaluating all turnes for topical relevance...")
        root_segments_data = [{"turn_id": c['turnID'], "content": self._format_turns(c['messages'][:4])} for c in conversations]
        tip_segments_data = [{"turn_id": c['turnID'], "content": self._format_turns(c['messages'][-4:])} for c in conversations]
        
        root_evals = self.evaluate_segments(user_prompt=user_prompt, segments_to_evaluate=str(root_segments_data)).evaluations
        tip_evals = self.evaluate_segments(user_prompt=user_prompt, segments_to_evaluate=str(tip_segments_data)).evaluations

        best_root_eval = max(root_evals, key=lambda x: x.relevance)
        best_tip_eval = max(tip_evals, key=lambda x: x.relevance)

        if best_root_eval.turn_id == best_tip_eval.turn_id:
            chosen_turn_id = best_root_eval.turn_id
            chosen_turn = next((c for c in conversations if c['turnID'] == chosen_turn_id), None)
            decision = ConnectionDecision(
                turn_id=chosen_turn_id,
                attach_at_turn_index=len(chosen_turn['messages']) - 1,
                reasoning=f"Both root and tip analysis converged on turn {chosen_turn_id}."
            )
            return {'decision': decision, 'score': best_tip_eval.relevance}
        else:
            # This conflict resolution could be improved, but we'll leave it for now
            conflict_decision = self.resolve_conflict(
                user_prompt=user_prompt,
                candidate_A_info=f"turn ID: {best_root_eval.turn_id}, Reasoning: {best_root_eval.reasoning}",
                candidate_B_info=f"turn ID: {best_tip_eval.turn_id}, Reasoning: {best_tip_eval.reasoning}"
            ).decision
            best_score = max(best_root_eval.relevance, best_tip_eval.relevance)
            return {'decision': conflict_decision, 'score': best_score}

In [59]:
def debug_print_tree(conversations: list):
    """
    An accurate debugger that understands the "taxonomy" data structure,
    where forked turnes only contain new messages.
    """
    if not conversations:
        print("[DEBUG] Tree is empty.")
        return

    print("\n" + "="*20 + " [DEBUG] Tree Structure " + "="*20)
    
    # Create a lookup map for turnes and their children
    turn_map = {b['turnID']: b for b in conversations}
    children_map = {}
    for turn_id, turn in turn_map.items():
        parent_id = turn.get('parent_turn_id')
        if parent_id is not None:
            if parent_id not in children_map:
                children_map[parent_id] = []
            children_map[parent_id].append(turn_id)

    # Find root nodes (those without a parent)
    root_ids = [b['turnID'] for b in conversations if b.get('parent_turn_id') is None]

    def print_turn_recursively(turn_id, indent=""):
        turn = turn_map.get(turn_id)
        if not turn: return
        
        # Print only the messages that physically exist in this turn
        for i, msg in enumerate(turn['messages']):
            # Determine the prefix for the line
            if i == 0:
                parent_id = turn.get('parent_turn_id')
                parent_turn = turn.get('parent_turn_index')
                if parent_id is not None:
                    # This is the first message of a forked turn
                    prefix = f"{indent}🌿 Fork from turn {parent_id}[{parent_turn}] ➜ Turn {i}"
                else:
                    # This is the first message of a root turn
                    prefix = f"{indent}🌿 Root turn {turn_id} ➜ Turn {i}"
            else:
                # Subsequent messages in the same turn
                prefix = f"{indent}{' ' * (len(str(turn_id)) + 14)}➜ Turn {i}"

            content_preview = (msg['content'][:60] + '...').replace('\n', ' ')
            print(f"{prefix}: {content_preview}")

        # Recurse for children
        if turn_id in children_map:
            for child_id in children_map[turn_id]:
                print_turn_recursively(child_id, indent + "  ")

    # Start printing from each root to show all separate trees
    for root_id in root_ids:
        print_turn_recursively(root_id)
        print("-" * 64)

In [6]:
# --- Signature for Generating a Response ---
class GenerateResponse(dspy.Signature):
    """Given a conversation history, continue the conversation by responding to the last user question."""
    history: str = dspy.InputField(desc="The history of the conversation so far.")
    question: str = dspy.InputField(desc="The user's latest question.")
    answer: str = dspy.OutputField(desc="A concise and helpful answer.")

In [10]:
class StatefulChat:
    RELEVANCE_THRESHOLD = 0.6

    def __init__(self, filepath="conversation_tree.json"):
        self.filepath = filepath
        self.router = ConversationRouter()
        self.responder = dspy.Predict(GenerateResponse)
        self.conversations = self._load()
        self.turn_map = {b['turnID']: b for b in self.conversations}
        # NEW: Track the last turn that was added or modified
        self.last_active_turn = self.turn_map.get(max(self.turn_map.keys())) if self.turn_map else None
        print(f"Chat manager initialized. Loaded {len(self.conversations)} turnes.")

    def _load(self):
        if not os.path.exists(self.filepath): return []
        with open(self.filepath, 'r', encoding='utf-8') as f: return json.load(f)

    def _save(self):
        with open(self.filepath, 'w', encoding='utf-8') as f: json.dump(self.conversations, f, ensure_ascii=False, indent=2)

    def _get_full_context(self, turn_id, turn_index) -> list:
        full_history = []
        curr_turn_id, curr_turn_index = turn_id, turn_index
        while curr_turn_id is not None:
            turn = self.turn_map.get(curr_turn_id)
            if not turn: break
            messages_in_segment = turn['messages'][:curr_turn_index + 1]
            full_history = messages_in_segment + full_history
            curr_turn_index = turn.get('parent_turn_index')
            curr_turn_id = turn.get('parent_turn_id')
        return full_history

    def _add_to_tree(self, decision, user_prompt, assistant_response):
        max_id = max([c['turnID'] for c in self.conversations] + [0])
        new_turn_id = max_id + 1
        
        new_turns = [{"role": "user", "content": user_prompt}, {"role": "assistant", "content": assistant_response}]
        
        if decision.attach_at_turn_index == -1: # New Root
            new_turn = { "turnID": new_turn_id, "parent_turn_id": None, "parent_turn_index": None, "messages": new_turns }
        else: # Fork
            new_turn = { "turnID": new_turn_id, "parent_turn_id": decision.turn_id, "parent_turn_index": decision.attach_at_turn_index, "messages": new_turns }
        
        self.conversations.append(new_turn)
        self.turn_map[new_turn_id] = new_turn
        # NEW: Update the last active turn reference
        self.last_active_turn = new_turn

    def chat(self, user_prompt: str):
        print(f"\n>> User: {user_prompt}")
        
        # MODIFIED: Pass the last_active_turn object to the router
        routing_result = self.router(
            user_prompt=user_prompt, 
            conversations=self.conversations,
            last_active_turn=self.last_active_turn 
        )
        decision, score = routing_result['decision'], routing_result['score']
        print(f"🧠 Router Score: {score:.2f}")

        # The threshold logic now correctly handles both topical relevance and forced follow-ups
        if score < self.RELEVANCE_THRESHOLD:
            print(f"⚠️ Score is below threshold. Creating a new root turn.")
            # Override decision to ensure a new root is created
            decision = ConnectionDecision(turn_id=0, attach_at_turn_index=-1, reasoning="Score below threshold, creating new topic.")
        else:
            print(f"✅ Score is above threshold. Following decision: {decision.reasoning}")
        
        context_messages = []
        if decision.attach_at_turn_index != -1:
            context_messages = self._get_full_context(decision.turn_id, decision.attach_at_turn_index)
        
        history_str = self.router._format_turns(context_messages)
        response = self.responder(history=history_str, question=user_prompt)
        assistant_response = response.answer
        print(f"<< Assistant: {assistant_response}")

        self._add_to_tree(decision, user_prompt, assistant_response)
        self._save()
        #print(f"💾 Tree updated and saved. Total turnes: {len(self.conversations)}")
        #debug_print_tree(self.conversations)
        return assistant_response

In [12]:
if os.path.exists("my_chat_tree.json"):
    os.remove("my_chat_tree.json")
    
chat_app = StatefulChat(filepath="my_chat_tree.json")

Chat manager initialized. Loaded 0 branches.


In [13]:
chat_app.chat("Help me understand gravity.")


>> User: Help me understand gravity.
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: No existing conversations.
<< Assistant: Gravity is the force that pulls any two pieces of matter toward each other. On Earth, it gives objects weight and keeps us on the ground. In space, it keeps planets orbiting the Sun and moons orbiting planets. The more mass something has, the stronger its gravity; the closer you are to it, the stronger the pull. Einstein later showed that gravity isn’t just a force but a curvature of space-time caused by mass.


'Gravity is the force that pulls any two pieces of matter toward each other. On Earth, it gives objects weight and keeps us on the ground. In space, it keeps planets orbiting the Sun and moons orbiting planets. The more mass something has, the stronger its gravity; the closer you are to it, the stronger the pull. Einstein later showed that gravity isn’t just a force but a curvature of space-time caused by mass.'

In [14]:
chat_app.chat("What's the difference between Newton's and Einstein's theories of gravity?")


>> User: What's the difference between Newton's and Einstein's theories of gravity?
🧠 Follow-up Detected: The new prompt directly references "Newton's and Einstein's theories of gravity," which were just discussed in the last turn. The question only makes sense in the context of the previous explanation about gravity and Einstein's refinement of the concept.
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Direct follow-up to the most recent turn. The new prompt directly references "Newton's and Einstein's theories of gravity," which were just discussed in the last turn. The question only makes sense in the context of the previous explanation about gravity and Einstein's refinement of the concept.
<< Assistant: Newton pictured gravity as an invisible force acting instantly between masses, with strength depending only on mass and distance. Einstein replaced that force with geometry: mass and energy curve the fabric of space-time, and objects follow the straightest p

'Newton pictured gravity as an invisible force acting instantly between masses, with strength depending only on mass and distance. Einstein replaced that force with geometry: mass and energy curve the fabric of space-time, and objects follow the straightest possible paths (geodesics) through that curved geometry. Newton’s theory works well for everyday speeds and weak fields, but Einstein’s general relativity predicts and explains phenomena Newton’s cannot—such as Mercury’s orbit, gravitational time dilation, and the bending of light by massive objects.'

In [15]:
chat_app.chat("Is gravity a force or something else?")


>> User: Is gravity a force or something else?
🧠 Follow-up Detected: The new prompt directly continues the topic of gravity and uses the pronoun 'it' implicitly referring to the concept just discussed. The question 'Is gravity a force or something else?' only makes sense in the context of the previous explanation contrasting Newton's force-based view with Einstein's geometric interpretation.
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Direct follow-up to the most recent turn. The new prompt directly continues the topic of gravity and uses the pronoun 'it' implicitly referring to the concept just discussed. The question 'Is gravity a force or something else?' only makes sense in the context of the previous explanation contrasting Newton's force-based view with Einstein's geometric interpretation.
<< Assistant: It depends on which theory you use. In Newton’s view, gravity is a force that acts between masses. In Einstein’s general relativity, gravity is not a for

'It depends on which theory you use. In Newton’s view, gravity is a force that acts between masses. In Einstein’s general relativity, gravity is not a force at all—it’s the curvature of space-time caused by mass and energy, and objects simply follow that curvature. So modern physics treats gravity as geometry, not a force.'

In [16]:
chat_app.chat("Give me a good recipe for a vegan pasta sauce.")


>> User: Give me a good recipe for a vegan pasta sauce.
🤔 No direct follow-up. Evaluating all branches for topical relevance...
🧠 Router Score: 0.00
⚠️ Score is below threshold. Creating a new root branch.
<< Assistant: Creamy Tomato-Basil Vegan Pasta Sauce  
Ingredients (serves 4):  
- 2 Tbsp olive oil  
- 4 cloves garlic, minced  
- 1 small onion, diced  
- 1 can (14 oz) crushed tomatoes  
- 1 cup raw cashews, soaked 30 min & drained  
- ½ cup unsweetened plant milk (soy/almond)  
- 2 Tbsp nutritional yeast  
- 1 tsp dried oregano  
- ½ tsp red-pepper flakes (optional)  
- Salt & pepper to taste  
- 1 packed cup fresh basil leaves, torn  

Steps:  
1. Sauté onion in olive oil over medium heat 4 min; add garlic 1 min more.  
2. Stir in tomatoes, oregano, pepper flakes; simmer 10 min.  
3. Blend cashews with plant milk until silky smooth.  
4. Pour cashew cream into sauce; simmer 5 min. Season.  
5. Off heat, fold in basil. Serve hot over pasta, garnished with extra basil.


'Creamy Tomato-Basil Vegan Pasta Sauce  \nIngredients (serves 4):  \n- 2 Tbsp olive oil  \n- 4 cloves garlic, minced  \n- 1 small onion, diced  \n- 1 can (14 oz) crushed tomatoes  \n- 1 cup raw cashews, soaked 30 min & drained  \n- ½ cup unsweetened plant milk (soy/almond)  \n- 2 Tbsp nutritional yeast  \n- 1 tsp dried oregano  \n- ½ tsp red-pepper flakes (optional)  \n- Salt & pepper to taste  \n- 1 packed cup fresh basil leaves, torn  \n\nSteps:  \n1. Sauté onion in olive oil over medium heat 4 min; add garlic 1 min more.  \n2. Stir in tomatoes, oregano, pepper flakes; simmer 10 min.  \n3. Blend cashews with plant milk until silky smooth.  \n4. Pour cashew cream into sauce; simmer 5 min. Season.  \n5. Off heat, fold in basil. Serve hot over pasta, garnished with extra basil.'

In [17]:
chat_app.chat("Who coined the word gravity?")


>> User: Who coined the word gravity?
🤔 No direct follow-up. Evaluating all branches for topical relevance...
🧠 Router Score: 0.00
⚠️ Score is below threshold. Creating a new root branch.
<< Assistant: Isaac Newton first used the word “gravity” in its modern scientific sense in his 1687 work *Philosophiæ Naturalis Principia Mathematica*.


'Isaac Newton first used the word “gravity” in its modern scientific sense in his 1687 work *Philosophiæ Naturalis Principia Mathematica*.'

In [18]:
chat_app.chat("How old was he?")


>> User: How old was he?
🧠 Follow-up Detected: The pronoun 'he' clearly refers to Isaac Newton, the subject of the previous turn, making this a direct contextual follow-up.
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Direct follow-up to the most recent turn. The pronoun 'he' clearly refers to Isaac Newton, the subject of the previous turn, making this a direct contextual follow-up.
<< Assistant: Isaac Newton was 44 or 45 years old when *Principia* was published in 1687 (born 25 December 1642, old style calendar).


'Isaac Newton was 44 or 45 years old when *Principia* was published in 1687 (born 25 December 1642, old style calendar).'

In [19]:
chat_app.chat("Where did he live?")


>> User: Where did he live?
🧠 Follow-up Detected: The pronoun "he" clearly refers to Isaac Newton, the subject of the previous turn, making this a direct contextual follow-up.
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Direct follow-up to the most recent turn. The pronoun "he" clearly refers to Isaac Newton, the subject of the previous turn, making this a direct contextual follow-up.
<< Assistant: He lived at Woolsthorpe Manor in Lincolnshire, England, and later in Cambridge while working at Trinity College.


'He lived at Woolsthorpe Manor in Lincolnshire, England, and later in Cambridge while working at Trinity College.'

In [20]:
chat_app.chat("For the recipe, I don't like onion can you improve")


>> User: For the recipe, I don't like onion can you improve
🤔 No direct follow-up. Evaluating all branches for topical relevance...
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Both root and tip analysis converged on Branch 4.
<< Assistant: Simply omit the onion. Start by warming the olive oil over medium heat and add the minced garlic right away; sauté just 30–45 seconds until fragrant, then continue with the recipe as written. The sauce will still be rich and flavorful thanks to the tomatoes, cashew cream, nutritional yeast, and fresh basil.


'Simply omit the onion. Start by warming the olive oil over medium heat and add the minced garlic right away; sauté just 30–45 seconds until fragrant, then continue with the recipe as written. The sauce will still be rich and flavorful thanks to the tomatoes, cashew cream, nutritional yeast, and fresh basil.'

In [22]:
chat_app.chat("you said Gravity is the force that pulls any two pieces of matter, can you show me the formula ")


>> User: you said Gravity is the force that pulls any two pieces of matter, can you show me the formula 
🤔 No direct follow-up. Evaluating all branches for topical relevance...
🧠 Router Score: 1.00
✅ Score is above threshold. Following decision: Both root and tip analysis converged on Branch 9.
<< Assistant: F = G · (m₁·m₂) / r²


'F = G · (m₁·m₂) / r²'

In [23]:
def visualize_unified_tree(conversations: list, filename: str = "conversation_unified_tree.html"):
    """
    Creates and displays a single, unified tree by connecting all root
    turnes to a universal "super root" node.
    """
    
    G = nx.DiGraph()
    
    # 1. ADD THE UNIVERSAL ROOT NODE
    G.add_node('UNIVERSAL_ROOT', label='All Conversations', shape='star', color='#C70039', size=25)
    
    # 2. Add all nodes from all turnes (same as before)
    for turn in conversations:
        turn_id = turn['turnID']
        for i, turn in enumerate(turn['messages']):
            node_id = f"{turn_id}_{i}"
            content = turn.get('content', '')
            label_text = (content[:97] + '...') if len(content) > 100 else content
            node_label = '\n'.join(textwrap.wrap(label_text, width=30))
            node_title = content
            node_shape = 'box'
            color = "#4E86E8" if turn['role'] == 'user' else "#D4A35D"
            G.add_node(node_id, label=node_label, title=node_title, shape=node_shape, color=color)

    # 3. Add all edges, including connections to the universal root
    for turn in conversations:
        turn_id = turn['turnID']
        # Connect turns within the same turn
        for i in range(len(turn['messages']) - 1):
            from_node, to_node = f"{turn_id}_{i}", f"{turn_id}_{i+1}"
            G.add_edge(from_node, to_node)
            
        parent_id = turn.get('parent_turn_id')
        if parent_id is not None:
            # This is a FORKED turn, connect it to its direct parent
            parent_turn = turn.get('parent_turn_index')
            fork_start_node = f"{turn_id}_0"
            parent_node = f"{parent_id}_{parent_turn}"
            if G.has_node(parent_node) and G.has_node(fork_start_node):
                 G.add_edge(parent_node, fork_start_node, color="#C5C5C5", dashes=True)
        else:
            # This is a ROOT turn, connect it to the UNIVERSAL_ROOT
            turn_start_node = f"{turn_id}_0"
            if G.has_node(turn_start_node):
                 G.add_edge('UNIVERSAL_ROOT', turn_start_node, color="#A9A9A9", dashes=True)

    # --- Hierarchical Layout and Display ---
    net = Network(height="800px", width="100%", notebook=True, directed=True, cdn_resources='in_line')
    net.from_nx(G)
    options = """
    {
      "layout": { "hierarchical": { "enabled": true, "direction": "UD", "sortMethod": "directed", "levelSeparation": 150, "nodeSpacing": 200 }},
      "physics": { "enabled": false }
    }
    """
    net.set_options(options)
    net.show(filename)
    print(f"🌳 Unified Hierarchical Tree graph saved to {filename}")


In [24]:
visualize_unified_tree(chat_app.conversations)

conversation_unified_tree.html
🌳 Unified Hierarchical Tree graph saved to conversation_unified_tree.html
